# MultiXrank results interpretation

## Summary of the previous steps
We propose to address the problem of identifying new symptoms associated with rare diseases with a network strategy. Our objective is to represent the data as multilayer networks and apply a network exploration algorithm called Random Walk with Restart (RWR) implemented in multiXrank.

Biological networks are widely use to represent various types of interactions between biological entities. The further allows integrating heterogeneous data into a common multilayer network framework.

We constructed a heterogeneous multilayer network composed of two layers:
* The Rare-X layer: The first layer contains data provided by the organizers of the Rare-X challenge, including rare disease names, patients within the cohort, and symptoms associated with each patient individual. By navigating this layer starting from the disease of interest used as seed, we can bypass the need for frequency calculations. Indeed, the symptoms associated with the highest scores when the RWR is applied from a given seed will be the most relevant to this disease. The scores for theses symptoms will be influenced by the number of patients nodes connecting to the symptoms from the seed disease node.
* The Orphanet layer: The second layer contains information from the Orphanet and Human Phenotype Ontology (HPO) databases, with nodes representing rare diseases and phenotypes. Edges connect diseases if they share at least one common causative gene, allowing the walker to propagate from one disease to another in the network. The walker can also spread to phenotypes via disease-phenotype edges, weighted based on the reported frequency of the phenotype for each disease. Through the HPO ontology (i.e. phenotype-phenotype associations), the walker can further navigate across the phenotypes. Note that the three types of edges present in this layer are all weighted.
* The walker can jump between the two layers using the available mappings between Rare-X diseases and Orphanet diseases, using the bipartite interactions.

The main idea is to perform RWR using each Rare-X disease as seed. MultiXrank allows us to compute RWR scores for both symptom nodes and phenotypes nodes with respect to the given Rare-X disease used as seed, while incorporating heterogeneous associations between entities in the multilayer network. By comparing the ranks of symptom nodes and phenotypes nodes, we can pinpoint symptoms that have a high ranking but do not receive a similarly high ranking for their equivalent phenotypes.

Below, we pinpoint some symptoms that seem to be related to phenotypes, and symptoms that appear to be novel symptoms associated to rare diseases. **However, please be aware that we are not clinicians or experts in this field, and therefore, our analysis should be approached with caution as it may contain errors and approximations.**

In [1]:
import pandas as pd

In [2]:
seedsfile = '../data/Diseases_names_and_seeds_numbering.tsv'
resultsdir = '../results_MultiXrank/results/'
topresults = 15

First, we extract seed numbers and their corresponding disease names.

In [3]:
seeds = pd.read_csv(seedsfile, header=0, sep='\t')
seeds.head(5)

,RARE-X,ORPHANET,SEED NUMBER
0,4H Leukodystrophy,4H leukodystrophy,1
1,8p-related disorders,8p inverted duplication/deletion syndrome,2
2,AHC (Alternating Hemiplegia of Childhood),Alternating hemiplegia of childhood,3
3,ARHGEF9-related disorders,None,4
4,CACNA1A related disorders,None,5


Then, we read each result files and store them into a dictionnary.

In [4]:
dico_results = dict()

for index, row in seeds.iterrows():
    seed = row[2]
    disease = row[0]
    resultfile = f'{resultsdir}/results_disease_{seed}.tsv'
    dico_results[disease] = pd.read_csv(resultfile, header=0, sep='\t')
    del dico_results[disease]['empty']

## Disease with Orphanet correspondance

There are 15 over 27 Rare-X diseases with Orphanet correspondance.

In [5]:
seeds_withOrpha = seeds[seeds['ORPHANET'] != "None"]
seeds_withOrpha

,RARE-X,ORPHANET,SEED NUMBER
0,4H Leukodystrophy,4H leukodystrophy,1
1,8p-related disorders,8p inverted duplication/deletion syndrome,2
2,AHC (Alternating Hemiplegia of Childhood),Alternating hemiplegia of childhood,3
8,CHOPS Syndrome,Cognitive impairment-coarse facies-heart defec...,9
9,Classic homocystinuria,Classic homocystinuria,10
10,DYRK1A Syndrome,DYRK1A-related intellectual disability syndrome,11
12,FOXP1 Syndrome,Intellectual disability-severe speech delay-mi...,13
14,KDM5C-related disorders,KDM5C-related syndromic X-linked intellectual ...,15
15,Kleefstra syndrome,Kleefstra syndrome,16
16,Koolen-de Vries Syndrome,Koolen-De Vries syndrome,17


We load MultiXrank results for each disease (used as seed) and store them into a dictionnary ``dico_results_withOrpha``.

In [6]:
dico_results_withOrpha = dict()

for index, row in seeds_withOrpha.iterrows():
    seed = row[2]
    disease = row[0]
    resultfile = f'{resultsdir}/results_disease_{seed}.tsv'
    dico_results_withOrpha[disease] = pd.read_csv(resultfile, header=0, sep='\t')
    del dico_results_withOrpha[disease]['empty']

These are the keys to access the dictionnary:

In [7]:
dico_results_withOrpha.keys()

dict_keys(['4H Leukodystrophy', '8p-related disorders', 'AHC (Alternating Hemiplegia of Childhood)', 'CHOPS Syndrome', 'Classic homocystinuria', 'DYRK1A Syndrome', 'FOXP1 Syndrome', 'KDM5C-related disorders', 'Kleefstra syndrome', 'Koolen-de Vries Syndrome', 'Malan Syndrome', 'Ogden Syndrome (NAA10)', 'Pallister-Killian mosaic syndrome (PKS)', 'SYNGAP1 related disorders', 'Wiedemann-Steiner Syndrome (WSS)'])

### 4H Leukodystrophy

In [8]:
dico_results_withOrpha['4H Leukodystrophy'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,4H leukodystrophy,0.737434,Orphanet_layer,ORPHA:289494,0.110714,4H leukodystrophy
RareX_layer,Osteoporosis_Symptom_Present,0.000657,Orphanet_layer,HP:0001251,0.000782,Ataxia
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000628,Orphanet_layer,HP:0001260,0.000781,Dysarthria
RareX_layer,Coordination_Issues_Symptom_Present,0.000625,Orphanet_layer,HP:0001332,0.000773,Dystonia
RareX_layer,PELHS Intellectual disability,0.000607,Orphanet_layer,HP:0000164,0.000769,Abnormality of the dentition
RareX_layer,Abnormal_Bladder_Symptom_Present,0.000593,Orphanet_layer,HP:0000545,0.000768,Myopia
RareX_layer,Nearsightedness_Symptom_Present,0.000486,Orphanet_layer,HP:0000044,0.000767,Hypogonadotropic hypogonadism
RareX_layer,PELHS Visual impairment (other than refractive errors),0.000445,Orphanet_layer,HP:0000668,0.000767,Hypodontia
RareX_layer,PELHS Wheelchair,0.000445,Orphanet_layer,HP:0006808,0.000767,Cerebral hypomyelination
RareX_layer,Hypotonia_Symptom_Present,0.000345,Orphanet_layer,HP:0004322,0.000585,Short stature


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Coordination Issues -> Ataxia
    Cognitive Impairment, Coordination Issues -> Cerebral hypomyelination 
    Nearsightedness -> Myopia

However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to 4H Leukodystrophy:

    Osteoporosis
    Intellectual disability
    Abnormal Bladder

### 8P Related Disorders

In [9]:
dico_results_withOrpha['8p-related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,8p inverted duplication/deletion syndrome,0.735164,Orphanet_layer,ORPHA:96092,0.110393,8p inverted duplication/deletion syndrome
RareX_layer,CSHQ Subscale 6: Parasomnias,0.000675,Orphanet_layer,HP:0001263,0.000427,Global developmental delay
RareX_layer,CSHQ Subscale 5: Night Wakings,0.000656,Orphanet_layer,HP:0001249,0.000426,Intellectual disability
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000597,Orphanet_layer,HP:0000750,0.000415,Delayed speech and language development
RareX_layer,Heart_Defect_Symptom_Present,0.000560,Orphanet_layer,HP:0001999,0.000415,Abnormal facial shape
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.000555,Orphanet_layer,HP:0010864,0.000414,"Intellectual disability, severe"
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.000530,Orphanet_layer,HP:0000463,0.000413,Anteverted nares
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.000530,Orphanet_layer,HP:0001256,0.000413,"Intellectual disability, mild"
RareX_layer,CSHQ Subscale 8: Daytime Sleepiness,0.000530,Orphanet_layer,HP:0000400,0.000411,Macrotia
RareX_layer,CSHQ Subscale 7: Sleep Disordered Breathing,0.000513,Orphanet_layer,HP:0000767,0.000411,Pectus excavatum


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Abnormal muscle function, Hypotonia -> Hypertonia, Infantile muscular hypotonia
    
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to 8H Related Disorders:

    Sleep habit related symptoms (CSHQ)
    Heart defect
    

### Alternating Hemiplegia of Childhood

In [10]:
dico_results_withOrpha['AHC (Alternating Hemiplegia of Childhood)'].head(topresults+9).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Alternating hemiplegia of childhood,0.734757,Orphanet_layer,ORPHA:2131,0.110490,Alternating hemiplegia of childhood
RareX_layer,Hypotonia_Symptom_Present,0.000744,Orphanet_layer,ORPHA:442835,0.000484,Non-specific early-onset epileptic encephalopathy
RareX_layer,Coordination_Issues_Symptom_Present,0.000670,Orphanet_layer,ORPHA:569,0.000462,Familial or sporadic hemiplegic migraine
RareX_layer,Flexible_Joints_Symptom_Present,0.000652,Orphanet_layer,ORPHA:2382,0.000454,Lennox-Gastaut syndrome
RareX_layer,Unusual_Movements_Symptom_Present,0.000550,Orphanet_layer,ORPHA:98758,0.000453,Spinocerebellar ataxia type 6
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000509,Orphanet_layer,ORPHA:97,0.000449,Familial paroxysmal ataxia
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000451,Orphanet_layer,ORPHA:71518,0.000448,Benign paroxysmal torticollis of infancy
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.000438,Orphanet_layer,ORPHA:71517,0.000434,Rapid-onset dystonia-parkinsonism
RareX_layer,CSHQ Subscale 5: Night Wakings,0.000390,Orphanet_layer,ORPHA:1171,0.000431,Cerebellar ataxia-areflexia-pes cavus-optic atrophy-sensorineural hearing loss syndrome
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.000390,Orphanet_layer,ORPHA:209967,0.000418,Episodic ataxia type 6


For this disease, top-scoring nodes from the Orphanet layer are diseases and not phenotypes. These diseases are related to AHC beacuse they share causative genes. Some of these diseases causes Ataxia, which is one of the top scored phenotype (e.g: Familial paroxysmal ataxia, Spinocerebellar ataxia type 6, Episodic ataxia type 6, ...). 

For this disease, we display more lines of the results table, to reveal supplementary phenotypes, since 9 diseases scored higher than phenotypes (`topresults+9`).

Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Coordination Issues -> Ataxia
    Constipation -> Gastrointestinal dysmotility, Abnormality of the gastrointestinal tract
    Abnormal Eye Movement -> Nystagmus
    
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to AHC:

    Flexible Joints
    Unusual Movements
    Sleep habit related symptoms (CSHQ)
    Hypotonia
    ...


### CHOPS Syndrome

In [11]:
dico_results_withOrpha['CHOPS Syndrome'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Cognitive impairment-coarse facies-heart defects-obesity-pulmonary involvement-short stature-skeletal dysplasia syndrome,0.733540,Orphanet_layer,ORPHA:444077,0.110172,Cognitive impairment-coarse facies-heart defects-obesity-pulmonary involvement-short stature-skeletal dysplasia syndrome
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000543,Orphanet_layer,HP:0001249,0.000361,Intellectual disability
RareX_layer,Short_Stature_Symptom_Present,0.000532,Orphanet_layer,HP:0004322,0.000361,Short stature
RareX_layer,Undergrowth_Symptom_Present,0.000530,Orphanet_layer,HP:0001263,0.000359,Global developmental delay
RareX_layer,Delivery_Complication_Symptom_Present,0.000522,Orphanet_layer,HP:0000463,0.000351,Anteverted nares
RareX_layer,Decreased_Fetal_Movement_Symptom_Present,0.000518,Orphanet_layer,HP:0001513,0.000349,Obesity
RareX_layer,Asthma_Symptom_Present,0.000516,Orphanet_layer,HP:0000664,0.000347,Synophrys
RareX_layer,Growth_Hormone_Deficiency_Symptom_Present,0.000516,Orphanet_layer,HP:0002553,0.000347,Highly arched eyebrow
RareX_layer,Respitory_Abnormality_Symptom_Present,0.000515,Orphanet_layer,HP:0011842,0.000347,Abnormal skeletal morphology
RareX_layer,Obesity_Symptom_Present,0.000514,Orphanet_layer,HP:0000527,0.000347,Long eyelashes


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Undergrowth, Short Stature, Growth Hormone Deficiency -> Global developmental delay
    Obesity -> Obesity
    Asthma, Respiratory Abnormality, Decreased Pulmonary Function, COPD -> Abnormality of the respiratory system
    Cognitive Impairment -> Hearing impairment

    
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to CHOPS Syndrome:

    Abnormal Muscle Function
    Delivery Complication, Decreased Fetal Movement 

### Classic Homocystinuria

In [12]:
dico_results_withOrpha['Classic homocystinuria'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Classic homocystinuria,0.741071,Orphanet_layer,ORPHA:394,0.111290,Classic homocystinuria
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.000811,Orphanet_layer,HP:0001249,0.000596,Intellectual disability
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.000779,Orphanet_layer,HP:0001166,0.000586,Arachnodactyly
RareX_layer,CSHQ Subscale 6: Parasomnias,0.000762,Orphanet_layer,HP:0000939,0.000585,Osteoporosis
RareX_layer,CSHQ Subscale 1: Bedtime Resistance,0.000687,Orphanet_layer,HP:0001083,0.000584,Ectopia lentis
RareX_layer,CSHQ Subscale 5: Night Wakings,0.000650,Orphanet_layer,HP:0004337,0.000584,Abnormality of amino acid metabolism
RareX_layer,CSHQ Subscale 8: Daytime Sleepiness,0.000604,Orphanet_layer,HP:0002757,0.000584,Recurrent fractures
RareX_layer,CSHQ Subscale 7: Sleep Disordered Breathing,0.000601,Orphanet_layer,HP:0001519,0.000584,Disproportionate tall stature
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.000585,Orphanet_layer,HP:0000678,0.000583,Dental crowding
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000488,Orphanet_layer,HP:0002650,0.000446,Scoliosis


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Heart Deffect, Abnormal EKG -> Hypertension, Arteriovenous malformation
    Bruxism, Feeding Difficulties -> Dental crowding (??)
    
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to Classic homocystinuria:

    Sleep habit related symptoms (CSHQ)
    Abnormal Muscle Function
    Cafe Au Lait Spots


### DYRK1A Syndrome

In [13]:
dico_results_withOrpha['DYRK1A Syndrome'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,DYRK1A-related intellectual disability syndrome,0.729522,Orphanet_layer,ORPHA:464306,0.109506,DYRK1A-related intellectual disability syndrome
RareX_layer,ASD_Symptom_Present,0.000841,Orphanet_layer,HP:0001249,0.000449,Intellectual disability
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.000776,Orphanet_layer,HP:0001263,0.000448,Global developmental delay
RareX_layer,Undergrowth_Symptom_Present,0.000738,Orphanet_layer,HP:0000252,0.000445,Microcephaly
RareX_layer,Unusual_Eye_Gaze_Symptom_Present,0.000711,Orphanet_layer,HP:0000750,0.000439,Delayed speech and language development
RareX_layer,Constipation_Symptom_Present,0.000641,Orphanet_layer,HP:0001999,0.000438,Abnormal facial shape
RareX_layer,Repetitive_Stereotypic_Behavior_Symptom_Present,0.000611,Orphanet_layer,HP:0011968,0.000436,Feeding difficulties
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.000533,Orphanet_layer,HP:0001288,0.000434,Gait disturbance
RareX_layer,Short_Stature_Symptom_Present,0.000512,Orphanet_layer,HP:0003086,0.000430,Acromesomelia
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000491,Orphanet_layer,HP:0001250,0.000340,Seizure


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    ASD, Repetitive Stereotypic Behavior  -> Autistic behavior
    Undergrowth, Short Stature -> Intrauterine growth retardation, Acromesomelia
    Abnormal Muscle Function -> Gait disturbance
    Esophagus Issues, Feeding Difficulties -> Feeding Difficulties
    
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to DYRK1A Syndrome:

    Sleep habit related symptoms (CSHQ)
    Unusual Eye Gaze, Abnormal Optic Nerve

### FOXP1 Syndrome

In [14]:
dico_results_withOrpha['FOXP1 Syndrome'].head(topresults+2).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Intellectual disability-severe speech delay-mild dysmorphism syndrome,0.733717,Orphanet_layer,ORPHA:391372,0.110258,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Coordination_Issues_Symptom_Present,0.000820,Orphanet_layer,ORPHA:52417,0.000504,MALT lymphoma
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000809,Orphanet_layer,ORPHA:585877,0.000491,nan
RareX_layer,Short_Attention_Span_Symptom_Present,0.000572,Orphanet_layer,HP:0000750,0.000394,Delayed speech and language development
RareX_layer,Impulsivity_Symptom_Present,0.000500,Orphanet_layer,HP:0002474,0.000386,Expressive language delay
RareX_layer,Abnormal_Eye_Movement_Symptom_Present,0.000480,Orphanet_layer,HP:0011220,0.000385,Prominent forehead
RareX_layer,Constipation_Symptom_Present,0.000412,Orphanet_layer,HP:0009088,0.000384,Speech articulation difficulties
RareX_layer,Hypotonia_Symptom_Present,0.000396,Orphanet_layer,HP:0000316,0.000296,Hypertelorism
RareX_layer,Temper_Tantrums_Symptom_Present,0.000395,Orphanet_layer,HP:0002019,0.000296,Constipation
RareX_layer,PELHS Intellectual disability,0.000393,Orphanet_layer,HP:0000486,0.000296,Strabismus


For this disease, top-scoring nodes from the Orphanet layer are diseases and not phenotypes. These diseases are related to FOXP1 Syndrome beacuse they share causative genes. 

For this disease, we display more lines of the results table, to reveal supplementary phenotypes, since 2 diseases scored higher than phenotypes (`topresults+2`).

Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Cognitive Impairment, ASD, Short Attention Span -> Autistic behavior, Delayed speach and language development, Expressive language delay, Speech articulation difficulties
    Abnormal Eye Movement, Unusual Eye Gaze -> Strabismus, Downslanted palpebral fissures
    Learning disability -> Intellectual disability
    Coordination Issues -> Motor delay
    
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to FOXP1 Syndrome:

    Impulsivity
    Consipation
    Hypotonia, Abnormal Muscle Function
    Sleep Anxiety

### KDM5C-related disorders

In [15]:
dico_results_withOrpha['KDM5C-related disorders'].head(topresults).style.hide(axis='index')


layer,node,score,layer.1,node.1,score.1,description
RareX_layer,KDM5C-related syndromic X-linked intellectual disability,0.731201,Orphanet_layer,ORPHA:85279,0.109746,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 6: Parasomnias,0.001076,Orphanet_layer,HP:0000028,0.001024,Cryptorchidism
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.001050,Orphanet_layer,HP:0000750,0.001023,Delayed speech and language development
RareX_layer,CSHQ Subscale 5: Night Wakings,0.000993,Orphanet_layer,HP:0010864,0.001022,"Intellectual disability, severe"
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.000985,Orphanet_layer,HP:0002232,0.001019,Patchy alopecia
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.000914,Orphanet_layer,HP:0000327,0.001017,Hypoplasia of the maxilla
RareX_layer,Abnormal_Bladder_Symptom_Present,0.000881,Orphanet_layer,HP:0001250,0.000779,Seizure
RareX_layer,CSHQ Subscale 1: Bedtime Resistance,0.000863,Orphanet_layer,HP:0004322,0.000778,Short stature
RareX_layer,CSHQ Subscale 8: Daytime Sleepiness,0.000740,Orphanet_layer,HP:0001257,0.000767,Spasticity
RareX_layer,CSHQ Subscale 7: Sleep Disordered Breathing,0.000676,Orphanet_layer,HP:0001347,0.000766,Hyperreflexia


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    Impulsivity -> Aggressive behavior
    Short Attention Span -> Delayed speech and language development, Intellectual disability
   
However, some higly scored Rare-X symptoms don't seem to have equivalents in the top phenotypes, and might be novel symptoms associated to KDM5C-related disorders:

    Sleep habit related symptoms (CSHQ)
    Abnormal Bladder
    Anxiety
    Cognitive Impairment

### Kleefstra syndrome

In [16]:
dico_results_withOrpha['Kleefstra syndrome'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Kleefstra syndrome,0.736385,Orphanet_layer,ORPHA:261494,0.110526,Kleefstra syndrome
RareX_layer,Constipation_Symptom_Present,0.000480,Orphanet_layer,HP:0001263,0.000386,Global developmental delay
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000479,Orphanet_layer,HP:0000316,0.000381,Hypertelorism
RareX_layer,Hypotonia_Symptom_Present,0.000467,Orphanet_layer,HP:0001252,0.000381,Hypotonia
RareX_layer,Coordination_Issues_Symptom_Present,0.000426,Orphanet_layer,HP:0000750,0.000377,Delayed speech and language development
RareX_layer,Abnormal_Eye_Movement_Symptom_Present,0.000384,Orphanet_layer,HP:0010864,0.000376,"Intellectual disability, severe"
RareX_layer,Feeding_Difficulties_Symptom_Present,0.000355,Orphanet_layer,HP:0000463,0.000375,Anteverted nares
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000321,Orphanet_layer,HP:0003196,0.000375,Short nose
RareX_layer,ASD_Symptom_Present,0.000315,Orphanet_layer,HP:0000280,0.000373,Coarse facial features
RareX_layer,Repetitive_Stereotypic_Behavior_Symptom_Present,0.000298,Orphanet_layer,HP:0000232,0.000373,Everted lower lip vermilion


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    - Hypotonia
    - Cognitive / Intellectual disabilities

Potential new symtom(s):

    - Constipation
    - Coordination issues
    - Abnormal eye movement

### Koolen-de Vries Syndrome

In [17]:
dico_results_withOrpha['Koolen-de Vries Syndrome'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Koolen-De Vries syndrome,0.736175,Orphanet_layer,ORPHA:96169,0.110520,Koolen-De Vries syndrome
RareX_layer,Hypotonia_Symptom_Present,0.000653,Orphanet_layer,HP:0001249,0.000388,Intellectual disability
RareX_layer,Abnormal_EEG_Symptom_Present,0.000636,Orphanet_layer,HP:0001263,0.000386,Global developmental delay
RareX_layer,Seizures_Symptom_Present,0.000570,Orphanet_layer,HP:0001252,0.000381,Hypotonia
RareX_layer,Undergrowth_Symptom_Present,0.000567,Orphanet_layer,HP:0000286,0.000378,Epicanthus
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000524,Orphanet_layer,HP:0000431,0.000376,Wide nasal bridge
RareX_layer,Coordination_Issues_Symptom_Present,0.000447,Orphanet_layer,HP:0000508,0.000376,Ptosis
RareX_layer,Immunodeficiency_Symptom_Present,0.000423,Orphanet_layer,HP:0000582,0.000373,Upslanted palpebral fissure
RareX_layer,Conductive_Hearing_Loss_Symptom_Present,0.000423,Orphanet_layer,HP:0000280,0.000373,Coarse facial features
RareX_layer,Autoimmune_Disorder_Symptom_Present,0.000420,Orphanet_layer,HP:0000581,0.000373,Blepharophimosis


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    - Hypotonia
    - Cognitive / Intellectual disabilities

Potential new symtom(s):

    - Abnormal EEG
    - Seizures

### Malan Syndrome

In [18]:
dico_results_withOrpha['Malan Syndrome'].head(topresults).style.hide(axis='index')


layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Malan overgrowth syndrome,0.729427,Orphanet_layer,ORPHA:420179,0.109544,Malan overgrowth syndrome
RareX_layer,General_Overgrowth_Symptom_Present,0.000919,Orphanet_layer,ORPHA:561,0.000816,Marshall-Smith syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000894,Orphanet_layer,ORPHA:447980,0.000806,19p13.3 microduplication syndrome
RareX_layer,Hypotonia_Symptom_Present,0.000892,Orphanet_layer,HP:0005616,0.000645,Accelerated skeletal maturation
RareX_layer,Tall_Stature_Symptom_Present,0.000825,Orphanet_layer,HP:0011220,0.000487,Prominent forehead
RareX_layer,Coordination_Issues_Symptom_Present,0.000823,Orphanet_layer,HP:0003100,0.000486,Slender long bone
RareX_layer,Abnormal_Eye_Movement_Symptom_Present,0.000819,Orphanet_layer,HP:0002119,0.000486,Ventriculomegaly
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000742,Orphanet_layer,HP:0000494,0.000485,Downslanted palpebral fissures
RareX_layer,CSHQ Subscale 6: Parasomnias,0.000532,Orphanet_layer,HP:0000486,0.000484,Strabismus
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.000495,Orphanet_layer,HP:0002342,0.000484,"Intellectual disability, moderate"


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    - Overgrowth
    - Cognitive / Intellectual disabilities

Potential new symtom(s):

    - Hypotonia
    - Coodination issues

### Ogden Syndrome (NAA10)

In [19]:
dico_results_withOrpha['Ogden Syndrome (NAA10)'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Ogden syndrome,0.721195,Orphanet_layer,ORPHA:276432,0.108315,Ogden syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000987,Orphanet_layer,ORPHA:568,0.000831,"Microphthalmia, Lenz type"
RareX_layer,Coordination_Issues_Symptom_Present,0.000981,Orphanet_layer,HP:0001263,0.000513,Global developmental delay
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000967,Orphanet_layer,HP:0002650,0.000510,Scoliosis
RareX_layer,Hypotonia_Symptom_Present,0.000880,Orphanet_layer,HP:0000473,0.000502,Torticollis
RareX_layer,CSHQ Subscale 6: Parasomnias,0.000823,Orphanet_layer,HP:0001290,0.000501,Generalized hypotonia
RareX_layer,Difficulty_Nonverbal_Behaviors_Symptom_Present,0.000775,Orphanet_layer,HP:0008897,0.000501,Postnatal growth retardation
RareX_layer,CSHQ Subscale 8: Daytime Sleepiness,0.000746,Orphanet_layer,HP:0002213,0.000501,Fine hair
RareX_layer,PELHS Intellectual disability,0.000734,Orphanet_layer,HP:0100840,0.000500,Aplasia/Hypoplasia of the eyebrow
RareX_layer,Issue_Muscle_Arms_Legs_Symptom_Present,0.000654,Orphanet_layer,HP:0010055,0.000499,Broad hallux


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:

    - Hypotonia

Potential new symtom(s):

    - Cognitive impairment
    - Coordination issues
    - Abnormal muscle function

### Pallister-Killian mosaic syndrome (PKS)

In [20]:
dico_results_withOrpha['Pallister-Killian mosaic syndrome (PKS)'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Tetrasomy 12p,0.735236,Orphanet_layer,ORPHA:884,0.110381,Tetrasomy 12p
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001066,Orphanet_layer,HP:0004322,0.000670,Short stature
RareX_layer,Hypotonia_Symptom_Present,0.001064,Orphanet_layer,HP:0001252,0.000663,Hypotonia
RareX_layer,Hypopigmentation_Symptom_Present,0.000794,Orphanet_layer,HP:0000343,0.000658,Long philtrum
RareX_layer,Hyperpigmentation_Symptom_Present,0.000698,Orphanet_layer,HP:0000470,0.000657,Short neck
RareX_layer,Coordination_Issues_Symptom_Present,0.000595,Orphanet_layer,HP:0000508,0.000657,Ptosis
RareX_layer,Constipation_Symptom_Present,0.000538,Orphanet_layer,HP:0008070,0.000656,Sparse hair
RareX_layer,Abnormal_EEG_Symptom_Present,0.000531,Orphanet_layer,HP:0010864,0.000656,"Intellectual disability, severe"
RareX_layer,Seizures_Symptom_Present,0.000530,Orphanet_layer,HP:0002750,0.000656,Delayed skeletal maturation
RareX_layer,Abnormal_Inside_Mouth_Symptom_Present,0.000524,Orphanet_layer,HP:0005692,0.000655,Joint hyperflexibility


Different Rare-X symptoms associated with high scores appear to be consistent with the phenotype terms also having a high score, i.e.:
    
    - Hypotonia
    - Cognitive / Intellectual disability

ORPHA:178469: Intellectual disability linked to other diseases because they share mutated gene (SYNGAP1). No symptoms retreived 

Potential new symtom(s):

    - Pigmentation issues
    - Coordination issues
    - Constipation
    - Abnormal EEG

### SYNGAP1 related disorders

In [21]:
dico_results_withOrpha['SYNGAP1 related disorders'].head(topresults).style.hide(axis='index')


layer,node,score,layer.1,node.1,score.1,description
RareX_layer,SYNGAP1-related developmental and epileptic encephalopathy,0.731299,Orphanet_layer,ORPHA:544254,0.109986,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,ASD_Symptom_Present,0.000873,Orphanet_layer,ORPHA:442835,0.000704,Non-specific early-onset epileptic encephalopathy
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000816,Orphanet_layer,ORPHA:1942,0.000701,Myoclonic-astatic epilepsy
RareX_layer,Unusual_Eye_Gaze_Symptom_Present,0.000764,Orphanet_layer,ORPHA:178469,0.000701,nan
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000737,Orphanet_layer,HP:0001249,0.000570,Intellectual disability
RareX_layer,Hypotonia_Symptom_Present,0.000735,Orphanet_layer,HP:0001263,0.000567,Global developmental delay
RareX_layer,Temper_Tantrums_Symptom_Present,0.000732,Orphanet_layer,HP:0000750,0.000558,Delayed speech and language development
RareX_layer,Aggressive_Violent_Behavior_Symptom_Present,0.000708,Orphanet_layer,HP:0002197,0.000548,Generalized-onset seizure
RareX_layer,Difficulty_Nonverbal_Behaviors_Symptom_Present,0.000679,Orphanet_layer,HP:0010832,0.000547,Abnormality of pain sensation
RareX_layer,Abnormal_EEG_Symptom_Present,0.000656,Orphanet_layer,HP:0001251,0.000422,Ataxia


ORPHA:178469: Intellectual disability linked to other diseases because they share mutated gene (SYNGAP1). No symptoms retreived associated for now in the Orphanet website.

The Rare-X symptoms with the highest scores appear to be consistent with phenotypes with a high score:

    - ASD (first) --> Autistic behavior (8th)
    - Hypotonia 
    - Intellectual disability 
    - Seizures

In Orphanet, symptoms are associated more with development whereas in the Rare-X layer, symtoms are associated with behavioral symtoms. 

### Wiedemann-Steiner Syndrome (WSS)

In [22]:
dico_results_withOrpha['Wiedemann-Steiner Syndrome (WSS)'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Wiedemann-Steiner syndrome,0.727514,Orphanet_layer,ORPHA:319182,0.109388,Wiedemann-Steiner syndrome
RareX_layer,Short_Stature_Symptom_Present,0.001023,Orphanet_layer,ORPHA:589534,0.000603,nan
RareX_layer,Undergrowth_Symptom_Present,0.000808,Orphanet_layer,ORPHA:589595,0.000603,nan
RareX_layer,Hypotonia_Symptom_Present,0.000735,Orphanet_layer,ORPHA:402017,0.000584,nan
RareX_layer,Cognitive_Impairment_Symptom_Present,0.000673,Orphanet_layer,ORPHA:585918,0.000584,nan
RareX_layer,Temper_Tantrums_Symptom_Present,0.000652,Orphanet_layer,ORPHA:98831,0.000584,nan
RareX_layer,Feeding_Difficulties_Symptom_Present,0.000649,Orphanet_layer,ORPHA:98835,0.000584,nan
RareX_layer,Aggressive_Violent_Behavior_Symptom_Present,0.000636,Orphanet_layer,HP:0000750,0.000382,Delayed speech and language development
RareX_layer,Constipation_Symptom_Present,0.000620,Orphanet_layer,HP:0000316,0.000292,Hypertelorism
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.000580,Orphanet_layer,HP:0001252,0.000290,Hypotonia


ORPHA:589534 / ORPHA:589595 / ORPHA:98831 / ORPHA:402017 / ORPHA:585918 / ORPHA:98835: Leukemia link to WSS because of the shared mutated gene (KMT2A). No symptoms retreived associated for now in the Orphanet website.

The Rare-X symptoms with the highest scores appear to be consistent with phenotypes with a high score:

    - Hypotonia --> Hypotonia

Potential new symtom(s):

    - Short stature and  Undergrowth. 

## Rare-X diseases without Orphanet correspondance

There are 12 diseases in the Rare-X dataset that do not match directly an Orphanet entry. For these 12 diseases, interpreting the node ranks in the Orphanet layer is more challenging. Indeed, to reach the Orphanet layer from a Rare-X disease without Orphanet correspondence, the walker must pass through mapped Rare-X/Orphanet diseases (i.e. throught one of the 15 diseases that are present in both layers) but cannot directly diffuse from the disease considered to associated phenotypes. Consequently, Orphanet disease with a correspondance in the Rare-X layer are more likely to present high scores.

We observe that the Wiedemann-Steiner syndrome is the most hihly scored in the Orphanet layer for seed Rare-X disease without Orphanet correspondance (see the results below). It's likely that this syndrome received high scores for most Rare-X diseases without Orphanet correspondence due to three main reasons:
    
* **Reachability from the Rare-X layer:** The Wiedemann-Steiner syndrome has a correspondance in the Orphanet layer, so the Orphanet node for the syndrome can easily be reached trhough the bipartite interactions connecting the two layers.
    
* **High representation in the input dataset:** The Wiedemann-Steiner syndrome is prominently represented in the Rare-X cohort, with a total of 78 patients diagnosed with this condition (second most represented disease in the cohort).

* **Numerous *unspecific* symptoms:** This syndrome exhibits numerous *unspecific* symptoms within the dataset. Those symptoms are also found in many other Rare-X diseases. For example, the "Short_Stature" symptom is associated with 21 out of the 27 diseases in the Rare-X layer, the "Undergrowth_Symptom_Present" is linked to 20 diseases, and "Hypotonia_Symptom_Present" is linked to 26 diseases, and so on. Therefore, the Wiedemann-Steiner syndrome node in the Rare-X layer can easily be reached through those *unspecific* symptoms. Once the syndrome has been reached in the Rare-X layer, it can diffuse to the Orphanet node via the bipartite interaction.

In summary, due to the limited number of Rare-X diseases mapped to Orphanet diseases, the Orphanet scores obtained for these 12 Rare-X diseases without mapping should be interpreted with caution. However, the scores of nodes from the Rare-X layer do not suffer from this limitation. Hence, it is still possible to interpret high-scoring symptom nodes from the Rare-X layer as symptoms associated to the seed disease.

In [23]:
seeds_withoutOrpha = seeds[seeds['ORPHANET'] == "None"]
seeds_withoutOrpha

,RARE-X,ORPHANET,SEED NUMBER
3,ARHGEF9-related disorders,None,4
4,CACNA1A related disorders,None,5
5,CASK-Related Disorders,None,6
6,CHAMP1 related disorders,None,7
7,CHD2 related disorders,None,8
11,FAM177A1 Associated Disorder,None,12
13,HUWE1-related disorders,None,14
18,MSL3 -related disorders,None,19
19,NARS1 genetic mutation,None,20
22,Ring14 and related disorders,None,23


We load multiXrank results for each disease (used as seed) and store them into a dictionnary ``dico_results_withoutOrpha``.

In [24]:
dico_results_withoutOrpha = dict()

for index, row in seeds_withoutOrpha.iterrows():
    seed = row[2]
    disease = row[0]
    resultfile = f'{resultsdir}/results_disease_{seed}.tsv'
    dico_results_withoutOrpha[disease] = pd.read_csv(resultfile, header=0, sep='\t')
    del dico_results_withoutOrpha[disease]['empty']

These are the keys to access the dictionnary:

In [25]:
dico_results_withoutOrpha.keys()

dict_keys(['ARHGEF9-related disorders', 'CACNA1A related disorders', 'CASK-Related Disorders', 'CHAMP1 related disorders', 'CHD2 related disorders', 'FAM177A1 Associated Disorder', 'HUWE1-related disorders', 'MSL3 -related disorders', 'NARS1 genetic mutation', 'Ring14 and related disorders', 'SETBP1-related disorder', 'STXBP1 related Disorders'])

### ARHGEF9-related disorders

In [26]:
dico_results_withoutOrpha['ARHGEF9-related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,ARHGEF9-related disorders,0.743399,Orphanet_layer,ORPHA:319182,0.000002,Wiedemann-Steiner syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001658,Orphanet_layer,ORPHA:544254,0.000001,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Coordination_Issues_Symptom_Present,0.001050,Orphanet_layer,ORPHA:391372,0.000001,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,ASD_Symptom_Present,0.001022,Orphanet_layer,ORPHA:420179,0.000001,Malan overgrowth syndrome
RareX_layer,PELHS Learning disability,0.001019,Orphanet_layer,ORPHA:261494,0.000001,Kleefstra syndrome
RareX_layer,Repetitive_Stereotypic_Behavior_Symptom_Present,0.001017,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,Temper_Tantrums_Symptom_Present,0.001016,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,Difficulty_Nonverbal_Behaviors_Symptom_Present,0.001013,Orphanet_layer,ORPHA:85279,0.000000,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,Unusual_Eye_Gaze_Symptom_Present,0.001012,Orphanet_layer,ORPHA:276432,0.000000,Ogden syndrome
RareX_layer,Difficulty_Making_Friends_Symptom_Present,0.001010,Orphanet_layer,ORPHA:289494,0.000000,4H leukodystrophy


### CACNA1A related disorders

In [27]:
dico_results_withoutOrpha['CACNA1A related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,CACNA1A related disorders,0.715411,Orphanet_layer,ORPHA:319182,0.000003,Wiedemann-Steiner syndrome
RareX_layer,Hypotonia_Symptom_Present,0.002439,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,Coordination_Issues_Symptom_Present,0.002368,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.001834,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Abnormal_Eye_Movement_Symptom_Present,0.001775,Orphanet_layer,ORPHA:2131,0.000002,Alternating hemiplegia of childhood
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.001625,Orphanet_layer,ORPHA:261494,0.000002,Kleefstra syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001597,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,CSHQ Subscale 6: Parasomnias,0.001554,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,CSHQ Subscale 1: Bedtime Resistance,0.001492,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.001482,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome


### CASK-Related Disorders

In [28]:
dico_results_withoutOrpha['CASK-Related Disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,CASK-Related Disorders,0.731210,Orphanet_layer,ORPHA:319182,0.000003,Wiedemann-Steiner syndrome
RareX_layer,Coordination_Issues_Symptom_Present,0.001746,Orphanet_layer,ORPHA:261494,0.000001,Kleefstra syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001605,Orphanet_layer,ORPHA:391372,0.000001,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Bruxism_Symptom_Present,0.001540,Orphanet_layer,ORPHA:420179,0.000001,Malan overgrowth syndrome
RareX_layer,PELHS Microcephaly (<5th percentile),0.001293,Orphanet_layer,ORPHA:544254,0.000001,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Hypotonia_Symptom_Present,0.001125,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,PELHS Intellectual disability,0.001033,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.001030,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,Hypertonia_Symptom_Present,0.000770,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome
RareX_layer,Esophagus_Issues_Symptom_Present,0.000755,Orphanet_layer,ORPHA:289494,0.000000,4H leukodystrophy


### CHAMP1 related disorders

In [29]:
dico_results_withoutOrpha['CHAMP1 related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,CHAMP1 related disorders,0.724997,Orphanet_layer,ORPHA:319182,0.000003,Wiedemann-Steiner syndrome
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.002444,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,CSHQ Subscale 6: Parasomnias,0.002430,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,CSHQ Subscale 1: Bedtime Resistance,0.002108,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.002034,Orphanet_layer,ORPHA:261494,0.000001,Kleefstra syndrome
RareX_layer,CSHQ Subscale 5: Night Wakings,0.001997,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,CSHQ Subscale 8: Daytime Sleepiness,0.001983,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.001863,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 7: Sleep Disordered Breathing,0.001639,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,Hypotonia_Symptom_Present,0.001357,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome


### CHD2 related disorders

In [30]:
dico_results_withoutOrpha['CHD2 related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,CHD2 related disorders,0.732959,Orphanet_layer,ORPHA:319182,0.000002,Wiedemann-Steiner syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001559,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Abnormal_EEG_Symptom_Present,0.001541,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Seizures_Symptom_Present,0.001537,Orphanet_layer,ORPHA:420179,0.000001,Malan overgrowth syndrome
RareX_layer,PELHS Autism/PDD,0.001140,Orphanet_layer,ORPHA:261494,0.000001,Kleefstra syndrome
RareX_layer,PELHS Intellectual disability,0.001079,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,ASD_Symptom_Present,0.001043,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,Difficulty_Making_Friends_Symptom_Present,0.001019,Orphanet_layer,ORPHA:85279,0.000000,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,Repetitive_Stereotypic_Behavior_Symptom_Present,0.000965,Orphanet_layer,ORPHA:276432,0.000000,Ogden syndrome
RareX_layer,Temper_Tantrums_Symptom_Present,0.000929,Orphanet_layer,ORPHA:96092,0.000000,8p inverted duplication/deletion syndrome


### FAM177A1 Associated Disorder

In [31]:
dico_results_withoutOrpha['FAM177A1 Associated Disorder'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,FAM177A1 Associated Disorder,0.734818,Orphanet_layer,ORPHA:319182,0.000003,Wiedemann-Steiner syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001848,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,Hypotonia_Symptom_Present,0.001838,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Coordination_Issues_Symptom_Present,0.001838,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.001818,Orphanet_layer,ORPHA:261494,0.000001,Kleefstra syndrome
RareX_layer,ASD_Symptom_Present,0.001817,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,Temper_Tantrums_Symptom_Present,0.001815,Orphanet_layer,ORPHA:96169,0.000001,Koolen-De Vries syndrome
RareX_layer,Impulsivity_Symptom_Present,0.001812,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,Short_Attention_Span_Symptom_Present,0.001811,Orphanet_layer,ORPHA:85279,0.000000,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,Abnormal_EEG_Symptom_Present,0.001803,Orphanet_layer,ORPHA:276432,0.000000,Ogden syndrome


### HUWE1-related disorders

In [32]:
dico_results_withoutOrpha['HUWE1-related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,HUWE1-related disorders,0.717779,Orphanet_layer,ORPHA:319182,0.000004,Wiedemann-Steiner syndrome
RareX_layer,Short_Stature_Symptom_Present,0.002077,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001974,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Hypotonia_Symptom_Present,0.001679,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.001596,Orphanet_layer,ORPHA:261494,0.000002,Kleefstra syndrome
RareX_layer,Constipation_Symptom_Present,0.001561,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,CSHQ Subscale 5: Night Wakings,0.001548,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,Coordination_Issues_Symptom_Present,0.001508,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 6: Parasomnias,0.001488,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.001478,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome


### MSL3-related disorders

In [33]:
dico_results_withoutOrpha['MSL3 -related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,MSL3 -related disorders,0.719639,Orphanet_layer,ORPHA:319182,0.000004,Wiedemann-Steiner syndrome
RareX_layer,Recurrent_Infections_Symptom_Present,0.002651,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,Immunodeficiency_Symptom_Present,0.002424,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.001751,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Hypotonia_Symptom_Present,0.001661,Orphanet_layer,ORPHA:261494,0.000002,Kleefstra syndrome
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.001642,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.001620,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,CSHQ Subscale 6: Parasomnias,0.001549,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,CSHQ Subscale 7: Sleep Disordered Breathing,0.001453,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 5: Night Wakings,0.001394,Orphanet_layer,ORPHA:96169,0.000001,Koolen-De Vries syndrome


### NARS1 genetic mutation

In [34]:
dico_results_withoutOrpha['NARS1 genetic mutation'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,NARS1 genetic mutation,0.718584,Orphanet_layer,ORPHA:319182,0.000003,Wiedemann-Steiner syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.002555,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Hypotonia_Symptom_Present,0.002550,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.002527,Orphanet_layer,ORPHA:261494,0.000002,Kleefstra syndrome
RareX_layer,Constipation_Symptom_Present,0.002515,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,Feeding_Difficulties_Symptom_Present,0.002499,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,Undergrowth_Symptom_Present,0.002470,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,Hypertonia_Symptom_Present,0.002458,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,Defecation_Issues_Symptom_Present,0.001751,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome
RareX_layer,Coordination_Issues_Symptom_Present,0.001677,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability


### Ring14 and related disorders

In [35]:
dico_results_withoutOrpha['Ring14 and related disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,Ring14 and related disorders,0.714740,Orphanet_layer,ORPHA:319182,0.000005,Wiedemann-Steiner syndrome
RareX_layer,Short_Stature_Symptom_Present,0.003591,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Undergrowth_Symptom_Present,0.003297,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Prematurity_Symptom_Present,0.001553,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,CSHQ Subscale 5: Night Wakings,0.001513,Orphanet_layer,ORPHA:261494,0.000002,Kleefstra syndrome
RareX_layer,CSHQ Subscale 3: Sleep Duration,0.001476,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,CSHQ Subscale 6: Parasomnias,0.001408,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,CSHQ Subscale 8: Daytime Sleepiness,0.001354,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 2: Sleep onset Delay,0.001136,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001103,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome


### SETBP1-related disorder

In [36]:
dico_results_withoutOrpha['SETBP1-related disorder'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,SETBP1-related disorder,0.714570,Orphanet_layer,ORPHA:319182,0.000004,Wiedemann-Steiner syndrome
RareX_layer,Hypotonia_Symptom_Present,0.003999,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Abnormal_Muscle_Function_Symptom_Present,0.003306,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Coordination_Issues_Symptom_Present,0.002249,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,Issue_Muscle_Arms_Legs_Symptom_Present,0.002222,Orphanet_layer,ORPHA:261494,0.000002,Kleefstra syndrome
RareX_layer,Amniotic_Fluid_Issue_Symptom_Present,0.002121,Orphanet_layer,ORPHA:2131,0.000002,Alternating hemiplegia of childhood
RareX_layer,Cognitive_Impairment_Symptom_Present,0.001853,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,Short_Attention_Span_Symptom_Present,0.001779,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,Impulsivity_Symptom_Present,0.001779,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome
RareX_layer,Delivery_Complication_Symptom_Present,0.001709,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability


### STWBP1 related Disorders

In [37]:
dico_results_withoutOrpha['STXBP1 related Disorders'].head(topresults).style.hide(axis='index')

layer,node,score,layer.1,node.1,score.1,description
RareX_layer,STXBP1 related Disorders,0.723015,Orphanet_layer,ORPHA:319182,0.000003,Wiedemann-Steiner syndrome
RareX_layer,Hypotonia_Symptom_Present,0.002125,Orphanet_layer,ORPHA:544254,0.000002,SYNGAP1-related developmental and epileptic encephalopathy
RareX_layer,Cognitive_Impairment_Symptom_Present,0.002066,Orphanet_layer,ORPHA:391372,0.000002,Intellectual disability-severe speech delay-mild dysmorphism syndrome
RareX_layer,Coordination_Issues_Symptom_Present,0.001808,Orphanet_layer,ORPHA:420179,0.000002,Malan overgrowth syndrome
RareX_layer,Abnormal_EEG_Symptom_Present,0.001614,Orphanet_layer,ORPHA:261494,0.000001,Kleefstra syndrome
RareX_layer,PELHS Intellectual disability,0.001587,Orphanet_layer,ORPHA:2131,0.000001,Alternating hemiplegia of childhood
RareX_layer,Seizures_Symptom_Present,0.001572,Orphanet_layer,ORPHA:464306,0.000001,DYRK1A-related intellectual disability syndrome
RareX_layer,CSHQ Subscale 6: Parasomnias,0.001497,Orphanet_layer,ORPHA:96092,0.000001,8p inverted duplication/deletion syndrome
RareX_layer,CSHQ Subscale 5: Night Wakings,0.001463,Orphanet_layer,ORPHA:85279,0.000001,KDM5C-related syndromic X-linked intellectual disability
RareX_layer,CSHQ Subscale 4: Sleep Anxiety,0.001349,Orphanet_layer,ORPHA:276432,0.000001,Ogden syndrome
